# Mobile Architecture Modules

By [Akshaj Verma](https://akshajverma.com)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Define Input Tensor

In [2]:
input_tensor = torch.rand(8, 64, 128, 128)

input_tensor_batchsize = input_tensor.shape[0]
input_tensor_channel = input_tensor.shape[1]
input_tensor_size = input_tensor.shape[2]

In [3]:
print(f"Input tensor batch size = {input_tensor_batchsize}")
print(f"Input tensor chanels = {input_tensor_channel}")
print(f"Input tensor size = {input_tensor_size, input_tensor_size}")

Input tensor batch size = 8
Input tensor chanels = 64
Input tensor size = (128, 128)


## [2016] Fire Module from SqueezeNet [[paper](https://arxiv.org/abs/1602.07360)]

In [4]:
class FireModule(nn.Module):
    def __init__(self, c_in, c_out, channel_factor):
        super(FireModule, self).__init__()
        self.c_in = c_in
        self.c_out = c_out
        self.channel_factor = channel_factor
        
        if self.c_in < self.channel_factor:
            raise ValueError("Input channel cannot be less than or equal to channel factor.")
        
        self.conv_squeeze_1x1 = self.conv_block(c_in=self.c_in, c_out=self.c_in//self.channel_factor, kernel_size=1, stride=1, padding=0)
        self.conv_expand_1x1 = self.conv_block(c_in=self.c_in//self.channel_factor, c_out=self.c_in, kernel_size=1, stride=1, padding=0)
        self.conv_expand_3x3 = self.conv_block(c_in=self.c_in//self.channel_factor, c_out=self.c_in, kernel_size=3, stride=1, padding=1)
        
        self.conv_expand_final = self.conv_block(c_in=2*self.c_in, c_out=self.c_out, kernel_size=1, stride=1, padding=0) 
        
    def forward(self, x):
        x = self.conv_squeeze_1x1(x)
        
        o1 = self.conv_expand_1x1(x)
        o2 = self.conv_expand_3x3(x)
        
        cat = torch.cat((o1, o2), dim = 1)
        out = self.conv_expand_final(cat)
        
        return out
    
    def conv_block(self, c_in, c_out, **kwargs):
        seq = nn.Sequential(
            nn.Conv2d(in_channels=c_in, out_channels=c_out, **kwargs),
            nn.BatchNorm2d(num_features=c_out),
            nn.ReLU()
        )
        
        return seq

In [5]:
fire_module = FireModule(c_in = input_tensor_channel, c_out = input_tensor_channel, channel_factor=4)

In [6]:
fire_module(input_tensor).shape

torch.Size([8, 64, 128, 128])

## [2017] MobileNet V1 [[paper](https://arxiv.org/abs/1704.04861)]

In [7]:
class MobilenetV1Module(nn.Module):
    def __init__(self, c_in, c_out, is_downsample):
        super(MobilenetV1Module, self).__init__()
        self.c_in = c_in
        self.c_out = c_out
        self.is_downsample = is_downsample
        
        self.conv_depthwise = self.depthwise_conv_block(c_in=self.c_in, c_out=self.c_in, is_downsample=self.is_downsample)
        self.conv_1x1 = self.conv_1x1_block(c_in = self.c_in, c_out = self.c_out)

    def forward(self, x):
        x = self.conv_depthwise(x)
        x = self.conv_1x1(x)
        
        return x
    
    def depthwise_conv_block(self, c_in, c_out, is_downsample):
        if is_downsample:
            seq = nn.Sequential(
                nn.Conv2d(in_channels=c_in, out_channels=c_out, kernel_size=3, stride=2, padding=1, groups=c_in),
                nn.BatchNorm2d(num_features=c_out),
                nn.ReLU6()
            )
        else:
            seq = nn.Sequential(
                nn.Conv2d(in_channels=c_in, out_channels=c_out, kernel_size=3, stride=1, padding=1, groups=c_in),
                nn.BatchNorm2d(num_features=c_out),
                nn.ReLU6()
            )

        return seq
    
    def conv_1x1_block(self, c_in, c_out):
        seq = nn.Sequential(
            nn.Conv2d(in_channels=c_in, out_channels=c_out, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(num_features=c_out),
            nn.ReLU6()
        )
        
        return seq

In [8]:
mobilenetv1_module = MobilenetV1Module(c_in=input_tensor_channel, c_out=input_tensor_channel, is_downsample=False)

In [9]:
mobilenetv1_module(input_tensor).shape

torch.Size([8, 64, 128, 128])

## [2018] MobileNet V2 [[paper](https://arxiv.org/abs/1801.04381)]

In [10]:
class MobilenetV2Module(nn.Module):
    def __init__(self, c_in, c_out, channel_factor, is_downsample):
        super(MobilenetV2Module, self).__init__()
        self.c_in = c_in
        self.c_out = c_out
        
        self.channel_factor = channel_factor
        self.is_downsample = is_downsample
        
        self.conv_expansion = self.conv_1x1_block(c_in = self.c_in, c_out = self.c_in * self.channel_factor, is_linear=False)
        
        self.conv_depthwise = self.depthwise_conv_block(c_in=self.c_in * self.channel_factor, 
                                                        c_out=self.c_in * self.channel_factor, 
                                                        is_downsample=self.is_downsample)
        
        self.conv_projection = self.conv_1x1_block(c_in = self.c_in * self.channel_factor, c_out = self.c_out, is_linear=True)

    def forward(self, x):
        o = self.conv_expansion(x)
        o = self.conv_depthwise(o)
        o = self.conv_projection(o)
        
        if self.c_in == self.c_out and self.is_downsample is False:
            o += x
        
        return o
    
    def depthwise_conv_block(self, c_in, c_out, is_downsample):
        if is_downsample:
            seq = nn.Sequential(
                nn.Conv2d(in_channels=c_in, out_channels=c_out, kernel_size=3, stride=2, padding=1, groups=c_in),
                nn.BatchNorm2d(num_features=c_out),
                nn.ReLU6()
            )
        else:
            seq = nn.Sequential(
                nn.Conv2d(in_channels=c_in, out_channels=c_out, kernel_size=3, stride=1, padding=1, groups=c_in),
                nn.BatchNorm2d(num_features=c_out),
                nn.ReLU6()
            )

        return seq
    
    def conv_1x1_block(self, c_in, c_out, is_linear):
        if is_linear:
            seq = nn.Sequential(
                nn.Conv2d(in_channels=c_in, out_channels=c_out, kernel_size=1, stride=1, padding=0),
                nn.BatchNorm2d(num_features=c_out),
            )
        else:
            seq = nn.Sequential(
                nn.Conv2d(in_channels=c_in, out_channels=c_out, kernel_size=1, stride=1, padding=0),
                nn.BatchNorm2d(num_features=c_out),
                nn.ReLU6()
            )
         
        
        return seq

In [11]:
mobilenetv2_module = MobilenetV2Module(c_in=input_tensor_channel, 
                                       c_out=input_tensor_channel, 
                                       channel_factor=2, 
                                       is_downsample=False)

In [12]:
mobilenetv2_module(input_tensor).shape

torch.Size([8, 64, 128, 128])

## [2018] SqNext Module from SqueezeNext [[paper](https://arxiv.org/abs/1803.10615)] 

In [16]:
class SqNextModule(nn.Module):
    def __init__(self, c_in, c_out):
        super(SqNextModule, self).__init__()
        self.c_in = c_in
        self.c_out = c_out
        
        self.conv_squeeze_1 = self.conv_block(c_in=self.c_in, c_out=self.c_in//2, kernel_size=1, stride=1, padding=0)
        self.conv_squeeze_2 = self.conv_block(c_in=self.c_in//2, c_out=self.c_in//4, kernel_size=1, stride=1, padding=0)
        
        self.conv_1 = self.conv_block(c_in=self.c_in//4, c_out=self.c_in//2, kernel_size=(1, 3), stride=1, padding=(0, 1))
        self.conv_2 = self.conv_block(c_in=self.c_in//2, c_out=self.c_in//2, kernel_size=(3, 1), stride=1, padding=(1, 0))
        
        self.conv_expand = self.conv_block(c_in=self.c_in//2, c_out=self.c_out, kernel_size=1, stride=1, padding=0)        
        
    def forward(self, x):
        o = self.conv_squeeze_1(x)
        o = self.conv_squeeze_2(o)
        o = self.conv_1(o)
        o = self.conv_2(o)
        o = self.conv_expand(o)
        
        if self.c_in == self.c_out:
            o += x
        
        return o
    
    def conv_block(self, c_in, c_out, **kwargs):
        seq = nn.Sequential(
            nn.Conv2d(in_channels=c_in, out_channels=c_out, **kwargs),
            nn.BatchNorm2d(num_features=c_out),
            nn.ReLU()
        )
        
        return seq

In [17]:
sqnext_module = SqNextModule(c_in=input_tensor_channel, c_out=input_tensor_channel*2)

In [18]:
sqnext_module(input_tensor).shape

torch.Size([8, 128, 128, 128])

## [2018] ShuffleNetV2 [[paper](https://arxiv.org/abs/1807.11164)]

In [8]:
class ShufflenetV2Module(nn.Module):
    def __init__(self, c_in, channel_factor, is_downsample):
        super(ShufflenetV2Module, self).__init__()
        self.c_in = c_in
        
        self.channel_factor = channel_factor
        self.is_downsample = is_downsample
        
        if self.is_downsample is False:
            self.conv_squeeze = self.conv_1x1_block(c_in = self.c_in//2, c_out=self.c_in//(2 * self.channel_factor))
            self.conv_depth = self.depthwise_conv_block(c_in = self.c_in//(2 * self.channel_factor), 
                                                        c_out = self.c_in//(2 * self.channel_factor),
                                                        is_downsample = self.is_downsample
                                                       )
            self.conv_expand = self.conv_1x1_block(c_in = self.c_in//(2 * self.channel_factor), c_out= self.c_in//2)
        else:
            # right
            self.conv_squeeze = self.conv_1x1_block(c_in = self.c_in, c_out=self.c_in//(self.channel_factor))
            self.conv_depth_1 = self.depthwise_conv_block(c_in = self.c_in//( self.channel_factor), 
                                                        c_out = self.c_in//(self.channel_factor),
                                                        is_downsample = self.is_downsample
                                                       )
            self.conv_expand = self.conv_1x1_block(c_in = self.c_in//(self.channel_factor), c_out= self.c_in)
            
            # left
            self.conv_depth_2 = self.depthwise_conv_block(c_in = self.c_in, 
                                                        c_out = self.c_in,
                                                        is_downsample = self.is_downsample
                                                       )
            self.conv_1x1 = self.conv_1x1_block(c_in = self.c_in, c_out= self.c_in)
    

    def forward(self, x):
        if self.is_downsample is False:
            x1, x2 = torch.split(x, split_size_or_sections=self.c_in//2, dim=1)
            
            x2 = self.conv_squeeze(x2)
            x2 = self.conv_depth(x2)
            x2 = self.conv_expand(x2)
            
            o = torch.cat((x1, x2), dim=1)
            o = o[:][torch.randperm(o.shape[0])][:, :]
            
        else:
            x1 = self.conv_squeeze(x)
            x1 = self.conv_depth_1(x1)
            x1 = self.conv_expand(x1)
            
            x2 = self.conv_depth_2(x)
            x2 = self.conv_1x1(x2)
            
            o = torch.cat((x1, x2), dim =1)
            o = o[:][torch.randperm(o.shape[0])][:, :]

        return o
        
    def depthwise_conv_block(self, c_in, c_out, is_downsample):
        if is_downsample:
            seq = nn.Sequential(
                nn.Conv2d(in_channels=c_in, out_channels=c_out, kernel_size=3, stride=2, padding=1, groups=c_in),
                nn.BatchNorm2d(num_features=c_out),
            )
        else:
            seq = nn.Sequential(
                nn.Conv2d(in_channels=c_in, out_channels=c_out, kernel_size=3, stride=1, padding=1, groups=c_in),
                nn.BatchNorm2d(num_features=c_out),
            )

        return seq
    
    def conv_1x1_block(self, c_in, c_out):
        seq = nn.Sequential(
            nn.Conv2d(in_channels=c_in, out_channels=c_out, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(num_features=c_out),
            nn.ReLU()
        )
        
        return seq

In [13]:
shufflenetv2_module = ShufflenetV2Module(c_in=input_tensor_channel, channel_factor=2, is_downsample=True)

In [14]:
shufflenetv2_module(input_tensor).shape

torch.Size([8, 128, 64, 64])

## [2019] MobileNet V3 [[paper](https://arxiv.org/abs/1905.02244)]

In [22]:
class MobilenetV3Module(nn.Module):
    def __init__(self, c_in, c_out, channel_factor):
        super(MobilenetV3Module, self).__init__()
        self.c_in = c_in
        self.c_out = c_out
        
        self.channel_factor = channel_factor
        
        self.conv_first = nn.Conv2d(in_channels=self.c_in, out_channels=self.c_in, kernel_size=3, stride=1, padding=1)
        self.depth_conv = nn.Conv2d(in_channels=self.c_in, out_channels=self.c_in, kernel_size=3, stride=1, padding=1, groups=self.c_in)
        
        self.squeeze = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        
        self.conv_excitation_downsample = nn.Conv2d(in_channels=self.c_in, 
                                               out_channels=self.c_in//self.channel_factor, 
                                               kernel_size=1, 
                                               stride=1, 
                                               padding=0)
        
        self.conv_excitation_upsample = nn.Conv2d(in_channels=self.c_in//self.channel_factor, 
                                             out_channels=self.c_in, 
                                             kernel_size=1, 
                                             stride=1, 
                                             padding=0)
        
        self.conv_last = nn.Conv2d(in_channels=self.c_in, out_channels=self.c_out, kernel_size=3, stride=1, padding=1)
        
        
        self.hswish = nn.Hardswish()
        self.relu = nn.ReLU()
        self.relu6 = nn.ReLU6()


    def forward(self, x):
        x = self.hswish(self.conv_first(x))
        x = self.hswish(self.depth_conv(x))
        
        o = self.squeeze(x)
        
        o = self.relu(self.conv_excitation_downsample(o))
        o = self.relu6(self.conv_excitation_upsample(o))
        

        x *= o
        x = self.conv_last(x)
        
        return x

In [54]:
mobilnetv3_module = MobilenetV3Module(c_in=input_tensor_channel, c_out=input_tensor_channel, channel_factor=2)

In [55]:
mobilnetv3_module(input_tensor).shape

torch.Size([8, 64, 128, 128])

## [2019] GhostModule from GhostNet [[paper](https://arxiv.org/abs/1911.11907)]

In [25]:
class GhostModule(nn.Module):
    def __init__(self, c_in):
        super(GhostModule, self).__init__()
        self.c_in = c_in
        
        self.conv = nn.Conv2d(in_channels=self.c_in, 
                              out_channels=self.c_in//2,
                              kernel_size=3,
                              stride=1,
                              padding=1)
        
        self.conv_depth = nn.Conv2d(in_channels=self.c_in//2, 
                                    out_channels=self.c_in//2, 
                                    kernel_size=3, 
                                    stride=1, 
                                    padding=1, 
                                    groups=self.c_in//2)

    def forward(self, x):
        x = self.conv(x)
        o = self.conv_depth(x)
        output = torch.cat((x, o), dim = 1)
        
        return output        

In [26]:
ghost_module = GhostModule(c_in=input_tensor_channel)

In [27]:
ghost_module(input_tensor).shape

torch.Size([8, 64, 128, 128])

In [15]:
class Fire(nn.Module):

    def __init__(
        self,
        inplanes: int,
        squeeze_planes: int,
        expand1x1_planes: int,
        expand3x3_planes: int
    ) -> None:
        super(Fire, self).__init__()
        self.inplanes = inplanes
        self.squeeze = nn.Conv2d(inplanes, squeeze_planes, kernel_size=1)
        self.squeeze_activation = nn.ReLU(inplace=True)
        self.expand1x1 = nn.Conv2d(squeeze_planes, expand1x1_planes,
                                   kernel_size=1)
        self.expand1x1_activation = nn.ReLU(inplace=True)
        self.expand3x3 = nn.Conv2d(squeeze_planes, expand3x3_planes,
                                   kernel_size=3, padding=1)
        self.expand3x3_activation = nn.ReLU(inplace=True)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.squeeze_activation(self.squeeze(x))
        return torch.cat([
            self.expand1x1_activation(self.expand1x1(x)),
            self.expand3x3_activation(self.expand3x3(x))
        ], 1)


class SqueezeNet(nn.Module):

    def __init__(
        self,
        version: str = '1_0',
        num_classes: int = 1000
    ) -> None:
        super(SqueezeNet, self).__init__()
        self.num_classes = num_classes
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
            Fire(64, 16, 64, 64),
            Fire(128, 16, 64, 64),
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
            Fire(128, 32, 128, 128),
            Fire(256, 32, 128, 128),
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
            Fire(256, 48, 192, 192),
            Fire(384, 48, 192, 192),
            Fire(384, 64, 256, 256),
            Fire(512, 64, 256, 256),
        )
    
        # Final convolution is initialized differently from the rest
        final_conv = nn.Conv2d(512, self.num_classes, kernel_size=1)
    
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            final_conv,
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if m is final_conv:
                    init.normal_(m.weight, mean=0.0, std=0.01)
                else:
                    init.kaiming_uniform_(m.weight)
                if m.bias is not None:
                    init.constant_(m.bias, 0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.classifier(x)
        return torch.flatten(x, 1)
